In [110]:
from math import sqrt,exp
import numpy as np
import matplotlib.pyplot as plt
import random
import math

In [111]:
def IJKScheme(M,N):
    r = 0
    K = 100
    T =1 
    dt = T/N
    row =0.36
    kappa = 2.58
    eta = 0.043
    v0 = 0.114
    s0 = 125
    sigma = 1

    c=[]

    for j in range(1,M):
        v =[]
        v.append(v0)
        s =[]
        s.append(s0)
        for i in range(1,N):
            x=random.gauss(0,1)
            randomTerm = row*x+sqrt(1-row**2)*x
            vtemp = max(v[i-1],0)
            firstTerm = kappa*(eta - vtemp)*dt
            secondTerm = sigma*sqrt(vtemp*dt)*randomTerm
            thirdTerm = 0.25*sigma*sigma*dt*(randomTerm**2 -1)
            v.append(v[i-1]+firstTerm + secondTerm + thirdTerm)
            sFirstTerm = (r - 0.25*(vtemp+v[i]))*dt
            sSecondTerm = row*sqrt(vtemp*dt)*randomTerm
            sThirdTerm = 0.5*(sqrt(max(v[i],0)) + sqrt(vtemp))*(x - row*randomTerm)*sqrt(dt)
            sFourthTerm = sigma*row*0.25*dt*(randomTerm**2 - 1)
            s.append(s[i-1] + sFirstTerm + sSecondTerm + sThirdTerm + sFourthTerm)
        
        c.append(math.exp(-r*T)*max(s[N-1]-K,0))
    
    price = (np.mean(c))
    error = (np.std(c)/math.sqrt(M))
    return price,error

In [112]:
priceIJK = []
errorIJK = []

for i in range(1,31,1):
    priceTemp,errorTemp = IJKScheme(1500,2000)
    priceIJK.append(priceTemp)
    errorIJK.append(errorTemp)

### Milstein's CI

In [27]:
def MilsteinScheme(M,N):
    r = 0
    K = 100
    T =1 
    dt = T/N
    row =0.36
    kappa = 2.58
    eta = 0.043
    v0 = 0.114
    s0 = 125
    sigma = 1

    c=[]

    for j in range(1,M):
        v=[]
        v.append(v0)
        s=[]
        s.append(s0)

        for i in range(1,N):
            x=random.gauss(0,1)
            randomTerm = row*x+math.sqrt(1-row**2)*x
            vtemp = max(0,v[i-1])
            firstTerm = kappa*(eta - vtemp*0.5)*dt
            thirdTerm = 0.25*(sigma**2)*dt*(randomTerm**2 - 1)
            secondTerm = sigma*((dt*(vtemp))**0.5)*randomTerm
            v.append(v[i-1]+ firstTerm + secondTerm + thirdTerm)
            s.append(s[i-1] + math.sqrt(vtemp*dt)*x + (r-vtemp*0.5)*dt)
        
        c.append(math.exp(-r*T)*max(s[N-1]-K,0))
    
    price = np.mean(c)       
    error = np.std(c)/math.sqrt(M)
    return price, error

In [28]:
priceMS = []
errorMS = []

for i in range(1,31,1):
    priceTemp,errorTemp = MilsteinScheme(1500,2000)
    priceMS.append(priceTemp)
    errorMS.append(errorTemp)

In [36]:
def EulerScheme(M,N):
    r = 0
    K = 100
    T =1 
    dt = T/N
    row =0.36
    kappa = 2.58
    eta = 0.043
    v0 = 0.114
    s0 = 125
    sigma = 1
    c=[]

    for j in range(1,M):
        v =[]
        v.append(v0)
        s =[]
        s.append(s0)
        for i in range(1,N):
            x=random.gauss(0,1)
            firstTerm = kappa*(eta - max(0,v[i-1]))*(dt)
            secondTerm = sigma*(max(0,v[i-1]) *dt)**0.5
            randomTerm = row*x+math.sqrt(1-row**2)*x
            v.append(max(0,v[i-1]) + firstTerm + secondTerm * randomTerm)
            s.append(s[i-1] + math.sqrt(max(v[i-1],0)*dt)*s[i-1]*x + r*s[i-1]*dt)
        
        c.append(math.exp(-r*T)*max(s[N-1]-K,0))
        
        
    price = np.mean(c)
    error = np.std(c)/math.sqrt(M)
    return price, error

In [37]:
priceES = []
errorES = []

for i in range(1,31,1):
    priceTemp,errorTemp = EulerScheme(1500,2000)
    priceES.append(priceTemp)
    errorES.append(errorTemp)

In [106]:
def RMSE(price, priceAct, n):
    pValue = scipy.stats.mstats.normaltest(price)[1]
    low,high = np.mean(price) + 2.58*(np.std(price)/sqrt(n)),np.mean(price) - 2.58*(np.std(price)/sqrt(n))
    interval = 2*2.58*(np.std(price)/sqrt(n))
    diffVar = []
    diff = []
    for i in range(0,30):
        diff.append(price[i]-priceAct)
        diffVar.append((price[i]-np.mean(price))**2)
    
    var = np.mean(diffVar)
    bias = np.mean(diff)
    rmse  = sqrt(bias**2 + var)
    print("The P-value of the Normality test is ", pValue)
    print("The width of the interval is", interval)
    print("The values range from ", low , "to ", high)
    print("The RMSE is ",rmse)
    return pValue,interval,low,high,rmse

In [113]:
RMSE(priceIJK,25.207,30)

The P-value of the Normality test is  0.542727313289
The width of the interval is 0.00773220213239
The values range from  25.1688591497 to  25.1611269475
The RMSE is  0.04280126196051082


(0.54272731328905355,
 0.0077322021323901757,
 25.168859149662673,
 25.161126947530285,
 0.04280126196051082)

In [114]:
RMSE(priceMS,25.207,30)

The P-value of the Normality test is  0.733342158151
The width of the interval is 0.00812447050184
The values range from  24.9235192554 to  24.9153947849
The RMSE is  0.2876722747819712


(0.73334215815076775,
 0.0081244705018390685,
 24.9235192554268,
 24.91539478492496,
 0.2876722747819712)

In [116]:
RMSE(priceES,25.61,30)

The P-value of the Normality test is  0.0274078799401
The width of the interval is 1.2972274378
The values range from  25.7057954146 to  24.4085679768
The RMSE is  1.4838048817827008


(0.027407879940106027,
 1.2972274378026787,
 25.705795414562388,
 24.408567976759713,
 1.4838048817827008)